In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [2]:
data=pd.read_csv('diabetes.csv')
df=data.copy()

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
y=df['Outcome']
X=df.drop('Outcome',axis=1)

In [8]:
from sklearn.preprocessing import RobustScaler

In [9]:
scaler = RobustScaler()
X_cols = X.columns
X = scaler.fit_transform(X)
X=pd.DataFrame(X,columns=X_cols)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=71,stratify=y)

In [11]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((614, 8), (614,), (154, 8), (154,))

In [12]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
xgb.score(X_train, y_train)


1.0

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n',cm)
print(
    '\nAccuracy score:',accuracy_score(y_test,y_pred),
    '\nPrecision score:',precision_score(y_test,y_pred),
    '\nRecall score:',recall_score(y_test,y_pred),
    '\nF1 score:',f1_score(y_test,y_pred),
)

Confusion Matrix
 [[81 19]
 [19 35]]

Accuracy score: 0.7532467532467533 
Precision score: 0.6481481481481481 
Recall score: 0.6481481481481481 
F1 score: 0.6481481481481481


<H1>Grid Search

In [14]:
# # Import and create Support Vector Classifier
# from sklearn.svm import SVC
# svc = SVC(random_state=1234)

# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# define parameters for Support Vector Classifier
xgb_param = {'gamma':[0.1, 0.3, 0.5, 0.6, 0.8, 1, 1.2, 1.4, 1.6], 
            'booster':['gbtree', 'gblinear'],
            'eta':[0.1, 0.3, 0.5, 0.6, 0.8, 1]
            }

# The parameters results in 7 x 2 x 5 = 70 different combinations
# CV=10 for 70 different combinations mean 700 jobs/model runs

xgb_grid = GridSearchCV(estimator=xgb, 
                        param_grid=xgb_param,
                        scoring='accuracy',
                        cv=10,
                        n_jobs=-1,
                        return_train_score=True)

# Fit the data to do Grid Search for Support Vector
xgb_grid_fit = xgb_grid.fit(X, y)

cv_results_xgb = pd.DataFrame.from_dict(xgb_grid_fit.cv_results_)

xgb_top_rank = cv_results_xgb[cv_results_xgb['rank_test_score'] == 1]

[19:05:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.



In [15]:
xgb_top_rank.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_eta,param_gamma,params,split0_test_score,split1_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
54,0.063855,0.020115,0.008102,0.004332,gblinear,0.1,0.1,"{'booster': 'gblinear', 'eta': 0.1, 'gamma': 0.1}",0.727273,0.766234,...,0.777135,0.778582,0.787265,0.782923,0.77424,0.777135,0.789017,0.774566,0.781105,0.005382
55,0.073419,0.018435,0.008228,0.006375,gblinear,0.1,0.3,"{'booster': 'gblinear', 'eta': 0.1, 'gamma': 0.3}",0.727273,0.766234,...,0.777135,0.778582,0.787265,0.782923,0.77424,0.777135,0.789017,0.774566,0.781105,0.005382
56,0.046885,0.009977,0.005585,0.003819,gblinear,0.1,0.5,"{'booster': 'gblinear', 'eta': 0.1, 'gamma': 0.5}",0.727273,0.766234,...,0.777135,0.778582,0.787265,0.782923,0.77424,0.777135,0.789017,0.774566,0.781105,0.005382
57,0.042094,0.011694,0.007391,0.005535,gblinear,0.1,0.6,"{'booster': 'gblinear', 'eta': 0.1, 'gamma': 0.6}",0.727273,0.766234,...,0.777135,0.778582,0.787265,0.782923,0.77424,0.777135,0.789017,0.774566,0.781105,0.005382
58,0.048101,0.014165,0.004697,0.001689,gblinear,0.1,0.8,"{'booster': 'gblinear', 'eta': 0.1, 'gamma': 0.8}",0.727273,0.766234,...,0.777135,0.778582,0.787265,0.782923,0.77424,0.777135,0.789017,0.774566,0.781105,0.005382


In [16]:
print('\n\n')
print('XGBoost')
print(xgb_top_rank['mean_test_score'])
print(xgb_top_rank['mean_train_score'])

# Print the best parameters of the Random Forest Classifier
print('\n The best Parameters are : ')
print(xgb_grid_fit.best_params_)





XGBoost
54     0.770882
55     0.770882
56     0.770882
57     0.770882
58     0.770882
59     0.770882
60     0.770882
61     0.770882
62     0.770882
63     0.770882
64     0.770882
65     0.770882
66     0.770882
67     0.770882
68     0.770882
69     0.770882
70     0.770882
71     0.770882
72     0.770882
73     0.770882
74     0.770882
75     0.770882
76     0.770882
77     0.770882
78     0.770882
79     0.770882
80     0.770882
81     0.770882
82     0.770882
83     0.770882
84     0.770882
85     0.770882
86     0.770882
87     0.770882
88     0.770882
89     0.770882
90     0.770882
91     0.770882
92     0.770882
93     0.770882
94     0.770882
95     0.770882
96     0.770882
97     0.770882
98     0.770882
99     0.770882
100    0.770882
101    0.770882
102    0.770882
103    0.770882
104    0.770882
105    0.770882
106    0.770882
107    0.770882
Name: mean_test_score, dtype: float64
54     0.781105
55     0.781105
56     0.781105
57     0.781105
58     0.781105
59     

In [17]:
xgb_opt = XGBClassifier(booster='gblinear',gamma=1,eta=0.1 )
xgb_opt.fit(X_train, y_train)

[19:05:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.



XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [18]:
yz_pred=xgb_opt.predict(X_test)

In [19]:
cm = confusion_matrix(y_test, yz_pred)
print('Confusion Matrix\n',cm)
print(
    '\nAccuracy score:',accuracy_score(y_test,yz_pred),
    '\nPrecision score:',precision_score(y_test,yz_pred),
    '\nRecall score:',recall_score(y_test,yz_pred),
    '\nF1 score:',f1_score(y_test,yz_pred),
)

Confusion Matrix
 [[88 12]
 [21 33]]

Accuracy score: 0.7857142857142857 
Precision score: 0.7333333333333333 
Recall score: 0.6111111111111112 
F1 score: 0.6666666666666666


In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n',cm)
print(
    '\nAccuracy score:',accuracy_score(y_test,y_pred),
    '\nPrecision score:',precision_score(y_test,y_pred),
    '\nRecall score:',recall_score(y_test,y_pred),
    '\nF1 score:',f1_score(y_test,y_pred),
)

Confusion Matrix
 [[81 19]
 [19 35]]

Accuracy score: 0.7532467532467533 
Precision score: 0.6481481481481481 
Recall score: 0.6481481481481481 
F1 score: 0.6481481481481481


<H1>Randomized Search

In [21]:

from sklearn.model_selection import RandomizedSearchCV

# define parameters for Random Forest
xgb_param = {'gamma':[0.1, 0.3, 0.5, 0.6, 0.8, 1, 1.2, 1.4, 1.6], 
            'booster':['gbtree', 'gblinear'],
            'eta':[0.1, 0.3, 0.5, 0.6, 0.8, 1]
            }


# Create the RandomizedSearchCV object
xgb_rs = RandomizedSearchCV(estimator=xgb, 
                        param_distributions=xgb_param,
                        scoring='accuracy',
                        cv=10,
                        n_iter=10,
                        return_train_score=True,
                        random_state=1234)

# n_iter selects 10 combinations out of 30 possible
# Now 10 x 10 = 100 jobs will be executed

# Fit the data to RandomizedSearchCV object
xgb_rs_fit = xgb_rs.fit(X_train, y_train)

# Get the results of RandomizedSearch
cv_results_xgb_rs = pd.DataFrame.from_dict(xgb_rs_fit.cv_results_)

# Print the best parameters of Randomized Search for Random Forest
print('\n The best Parameters are : ')
print(xgb_rs_fit.best_params_)

[19:05:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:04] WARNING: C:/bu

[19:05:06] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:06] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:06] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:06] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:06] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.

[19:05:06] WARNING: C:/bu

In [22]:
xgb_opt_random = XGBClassifier(booster='gblinear',gamma=0.6,eta=0.1 )
xgb_opt_random.fit(X_train, y_train)

[19:05:12] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma" } are not used.



XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=0.6, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [23]:
yz_pred_Random=xgb_opt_random.predict(X_test)

In [24]:
cm = confusion_matrix(y_test, yz_pred_Random)
print('Confusion Matrix\n',cm)
print(
    '\nAccuracy score:',accuracy_score(y_test,yz_pred_Random),
    '\nPrecision score:',precision_score(y_test,yz_pred_Random),
    '\nRecall score:',recall_score(y_test,yz_pred_Random),
    '\nF1 score:',f1_score(y_test,yz_pred_Random),
)

Confusion Matrix
 [[88 12]
 [21 33]]

Accuracy score: 0.7857142857142857 
Precision score: 0.7333333333333333 
Recall score: 0.6111111111111112 
F1 score: 0.6666666666666666
